In [2]:
from Pianki.Analiza_pianek import saldo, naliczone, wstrzymane
# from Pianki.Analiza_pianek.przygotowanie_danych import aktualizuj_saldo

# aktualizuj_saldo()
# from Modele_db import engine

# with engine.begin() as conn:
#     saldo.rename(columns={"KOD": "kod", "SALDO": "stan"}).to_sql("SALDO", conn, if_exists="replace")
#     naliczone.rename(columns={'LIMIT_NAZWA':"limit_nazwa", 'KOD':"kod", 'ZAPOT_ZLEC':"zapot_zlec", 'LIMIT_DATA_PROD':"limit_data_prod"}).to_sql("NALICZONE", conn, if_exists="replace")
#     wstrzymane.rename(columns={"KOD":"kod", 'ILOSC':"ilosc"}).to_sql("WSTRZYMANE", conn, if_exists="replace")

In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [ ]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

In [7]:
from Modele_db.modele_db import session, ZAM_PIANKI, KOMPLETY_PIANEK
from sqlalchemy import func


session.query(func.sum(ZAM_PIANKI.ile_zam * KOMPLETY_PIANEK.obj)).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(ZAM_PIANKI.data_zakonczenia.between(dt(2024,9,29), dt(2024,10,6))).all()


[(252.6100000000001,)]

In [7]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"]).drop_duplicates("OPIS")
# br.to_excel("BRAKI.xlsx")

In [ ]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

In [8]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)

In [ ]:
ava = arp["AVANT"].ar
ava.DO_ZAM_SZT.mean()

In [ ]:
analiza[analiza.OPIS.str.contains("WIL")][['index', 'KOD', 'OPIS', 'MAX','SALDO',
       'ZLECENIA', 'WST', 'CZEKA_NA_SPAKOWANIE', 'ZAMOWIONE',
       'CZESIOWO_DOSTARCZONE', '29/01', '30/01', 'MIN', 'SUMA_ZLEC',
       'SALDO_Z_NIE_SPAK', 'WOLNE_SALDO', 'WOLNE_NIE_SPAK', 'DO_ZAM_SZT',
       ]]

In [ ]:
ava[ava.DO_ZAM_SZT > ava.MIN]

In [ ]:
(arp > 1).query("GRUPA < 4")

In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [ ]:
arp["STONE"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [ ]:
arp._Tabela_podsumowania_analizy.columns

In [4]:
Zapis_danych_do_Archiwum(243m, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")